In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch

import pickle

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

c:\Users\Revalda Putawara\.conda\envs\bnntest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda")

In [4]:
class BayesianCNNSingleFC(PyroModule):
    def __init__(self, num_classes):
        super().__init__()

        prior_mu = 0.
        #prior_sigma = 0.1 #accuracy 13.203704% 2 epochs
        #prior_sigma = 1. #accuracy 31% 2 epochs
        prior_sigma = torch.tensor(10., device=device) #accuracy 45% 10 epochs
        #prior_sigma = 100 #accuracy 21% 10 epochs

        self.conv1 = PyroModule[nn.Conv2d](3, 32, kernel_size=5, stride=1, padding=2)
        self.conv1.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([32, 3, 5, 5]).to_event(4))
        self.conv1.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([32]).to_event(1))

        self.conv2 = PyroModule[nn.Conv2d](32, 64, kernel_size=5, stride=1, padding=2) #initially padding=1 kernel_size=3, without stride
        self.conv2.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([64, 32, 5, 5]).to_event(4))
        self.conv2.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([64]).to_event(1))

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = PyroModule[nn.Linear](64 * 16 * 16, num_classes)
        self.fc1.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([num_classes, 64 * 16 * 16]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([num_classes]).to_event(1))

    def forward(self, x, y=None):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        logits = self.fc1(x)
        
        # THIS IS THE MISSING PIECE: Define the likelihood
        if y is not None:
            with pyro.plate("data", x.shape[0]):
                pyro.sample("obs", dist.Categorical(logits=logits), obs=y)
        
        return logits

In [5]:
def load_data(batch_size=54):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3444, 0.3809, 0.4082], std=[0.1809, 0.1331, 0.1137])
    ])

    dataset = datasets.EuroSAT(root='./data', transform=transform, download=False)

    torch.manual_seed(42)
    
    with open('datasplit/split_indices.pkl', 'rb') as f:
        split = pickle.load(f)
        train_dataset = Subset(dataset, split['train'])
        test_dataset = Subset(dataset, split['test'])

    # Add num_workers and pin_memory for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

In [6]:
from pyro.infer.autoguide import AutoDiagonalNormal

In [7]:
num_classes = 10
bayesian_model = BayesianCNNSingleFC(num_classes=num_classes).to(device)

In [8]:
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
#guide.load_state_dict(torch.load(guide_path))

pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

In [9]:
from tqdm import tqdm

In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix


def predict_data_probs(model, test_loader, num_samples=10):
    model.eval()

    all_labels = []
    all_predictions = []
    all_logits = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            logits_mc = torch.zeros(num_samples, images.size(0), model.fc1.out_features).to(device)

            for i in range(num_samples):
                guide_trace = pyro.poutine.trace(guide).get_trace(images)
                replayed_model = pyro.poutine.replay(model, trace=guide_trace)
                logits = replayed_model(images)
                logits_mc[i] = logits

            avg_logits = logits_mc.mean(dim=0)
            predictions = torch.argmax(avg_logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_logits.extend(avg_logits.cpu().numpy())
            all_probs.extend(F.softmax(avg_logits, dim=1).cpu().numpy())

    return all_labels, all_predictions, all_logits, all_probs

In [179]:
train_loader, test_loader = load_data(batch_size=54)

In [180]:
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    print(value)

AutoDiagonalNormal.loc: torch.Size([217546])
Parameter containing:
tensor([100.0000,  -2.4763,  -1.0711,  ...,  -2.4452,   4.6454,   1.5156],
       device='cuda:0', requires_grad=True)
AutoDiagonalNormal.scale: torch.Size([217546])
tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)


In [181]:
all_labels, all_predictions, all_logits, all_probs = predict_data_probs(bayesian_model, test_loader, num_samples=10)

Evaluating: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


In [182]:
cm = confusion_matrix(all_labels, all_predictions)

In [183]:
#print accuracy from confusion matrix
accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy from confusion matrix: {accuracy * 100:.6f}%")

Accuracy from confusion matrix: 43.888889%


In [16]:
param_shapes = {
    "conv1.weight": (32, 3, 5, 5),
    "conv1.bias": (32,),
    "conv2.weight": (64, 32, 5, 5),
    "conv2.bias": (64,),
    "fc1.weight": (num_classes, 64 * 16 * 16),
    "fc1.bias": (num_classes,)
}

def _unpack(vector):
    """Unpacks flat vector into a dict of shaped tensors"""
    params = {}
    offset = 0
    for name, shape in param_shapes.items():
        size = torch.tensor(shape).prod().item()
        flat_param = vector[offset:offset + size]
        params[name] = flat_param.view(shape)
        offset += size
    return params

In [26]:

# access the value of generator pyro.get_param_store().items()
#pyro_params = pyro.get_param_store().items()


In [32]:
weight_dict = {}

for name, value in pyro.get_param_store().items():
    #print(f"{name}: {value.shape}")
    #print(value)
    weight_dict[name] = value.cpu().detach().numpy()

In [33]:
weight_dict

{'AutoDiagonalNormal.loc': array([ 3.148341 , -2.4762936, -1.0711346, ..., -2.4451642,  4.6454   ,
         1.5156032], shape=(217546,), dtype=float32),
 'AutoDiagonalNormal.scale': array([0.04540997, 0.03850836, 0.04395538, ..., 7.709083  , 6.1614437 ,
        6.695009  ], shape=(217546,), dtype=float32)}

In [ ]:
param_shapes = {
    "conv1.weight": (32, 3, 5, 5),
    "conv1.bias": (32,),
    "conv2.weight": (64, 32, 5, 5),
    "conv2.bias": (64,),
    "fc1.weight": (num_classes, 64 * 16 * 16),
    "fc1.bias": (num_classes,)
}

In [64]:
unpacked_params_loc = {}
offset_loc = 0

for name, shape in param_shapes.items():
    size = torch.tensor(shape).prod().item()
    flat_param = weight_dict["AutoDiagonalNormal.loc"][offset_loc:offset_loc + size]
    unpacked_params_loc[name] = torch.tensor(flat_param).view(shape)
    offset_loc += size

In [65]:
unpacked_params_scale = {}
offset_scale = 0

for name, shape in param_shapes.items():
    size = torch.tensor(shape).prod().item()
    flat_param = weight_dict["AutoDiagonalNormal.scale"][offset_scale:offset_scale + size]
    unpacked_params_scale[name] = torch.tensor(flat_param).view(shape)
    offset_scale += size

In [69]:
torch.cat([param.view(-1) for param in unpacked_params_loc.values()]).shape
torch.cat([param.view(-1) for param in unpacked_params_scale.values()]).shape

torch.Size([217546])

In [68]:
# modify the first item of unpacked_params_weights["conv1.weight"][0, 0, 0, 0]
unpacked_params_loc["conv1.weight"][0, 0, 0, 0] = 100.0
print(unpacked_params_loc["conv1.weight"][0, 0, 0, 0])

tensor(100.)


In [71]:
# return unpacked_params back to the param store, into a flattened form
# put unpacked_params_weights to pyro param store autodiagonalnormal.loc
# put unpacked_params_bias to pyro param store autodiagonalnormal.scale
unpacked_params_loc_flat = torch.cat([param.view(-1) for param in unpacked_params_loc.values()])
unpacked_params_scale_flat = torch.cat([param.view(-1) for param in unpacked_params_scale.values()])

#for name, param in unpacked_params_loc.items():
#    pyro.get_param_store().set_param(name, param)

In [ ]:
unpacked_params_scale_flat.shape

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950])

In [78]:
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    print(value)

AutoDiagonalNormal.loc: torch.Size([217546])
Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)
AutoDiagonalNormal.scale: torch.Size([217546])
tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)


In [80]:
param_store = pyro.get_param_store()
weight_loc = param_store["AutoDiagonalNormal.loc"]

In [81]:
weight_loc

Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

In [84]:
def bitflip_tensor(tensor, bit_index=0):
    flat_tensor = tensor.flatten()
    raw_bytes = flat_tensor.view(torch.uint8)

    # Choose a random byte and bit to flip
    byte_idx = torch.randint(0, raw_bytes.numel(), (1,)).item()
    mask = 1 << bit_index
    raw_bytes[byte_idx] ^= mask  # Flip bit

    return tensor

In [86]:
def flip_bit_at(tensor, index, bit_position=0):
    val = tensor.view(torch.uint8)
    val[index] ^= (1 << bit_position)
    return val.view(tensor.dtype)

In [97]:
# clean pyro param store
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
#guide.load_state_dict(torch.load(guide_path))

pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

In [108]:
# Flip bit at index 0, bit position 0 (least significant bit)
original_tensor = param_store["AutoDiagonalNormal.loc"]
flipped_tensor = flip_bit_at(original_tensor, index=0, bit_position=7)

# Update the parameter store with the modified tensor
param_store["AutoDiagonalNormal.loc"].data.copy_(flipped_tensor)

tensor([ 3.1484, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0')

In [105]:
pyro.get_param_store()["AutoDiagonalNormal.loc"].mean()

tensor(-0.1290, device='cuda:0', grad_fn=<MeanBackward0>)

In [92]:
param_store["AutoDiagonalNormal.loc"].data.copy_(flip_bit_at(param_store["AutoDiagonalNormal.loc"], 30, 0))

tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0')

In [137]:
param_store["AutoDiagonalNormal.loc"].data.copy_(bitflip_tensor(param_store["AutoDiagonalNormal.loc"]))

tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', grad_fn=<CopyBackwards>)

In [111]:
def flip_bit_in_tensor(tensor, bit_position=1, flip_count=1):
    flat = tensor.view(-1)
    idx = torch.randint(0, flat.numel(), (flip_count,))
    for i in idx:
        val = flat[i].item()
        int_val = np.frombuffer(np.float32(val).tobytes(), dtype=np.uint32)[0]
        flipped = int_val ^ (1 << bit_position)
        flipped_val = np.frombuffer(np.uint32(flipped).tobytes(), dtype=np.float32)[0]
        flat[i] = torch.tensor(flipped_val)
    return tensor

def inject_seu_conv_layer(layer, bit_position=10, flip_count=1):
    with torch.no_grad():
        layer.weight.data = flip_bit_in_tensor(layer.weight.data.clone(), bit_position, flip_count)

In [122]:
with torch.no_grad():
    flip_bit_in_tensor(pyro.get_param_store()["AutoDiagonalNormal.loc"], bit_position=3, flip_count=1)

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

RuntimeError: values expected sparse tensor layout but got Strided

## BEFORE BITFLIP

In [170]:
import copy

In [172]:
# clean pyro param store
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

original_param_store = {}

for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)

AutoDiagonalNormal.loc: torch.Size([217546])
AutoDiagonalNormal.scale: torch.Size([217546])


C:\Users\Revalda Putawara\AppData\Local\Temp\ipykernel_18828\1415134443.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)


In [173]:
original_param_store["AutoDiagonalNormal.loc"]

tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

## BITFLIP

In [ ]:
#with torch.no_grad():
#    flip_bit_in_tensor(pyro.get_param_store()["AutoDiagonalNormal.loc"], bit_position=3, flip_count=1)

In [174]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

In [175]:
# change the first item of pyro.get_param_store()["AutoDiagonalNormal.loc"] to 100
# pyro.get_param_store()["AutoDiagonalNormal.loc"][0] = 100.0

def change_first_item(param_store, new_value):
    pyro.get_param_store()[param_store][0] = new_value

    return pyro.get_param_store()[param_store]

# Change the first item with no gradient tracking
with torch.no_grad():
    param_store["AutoDiagonalNormal.loc"].data.copy_(change_first_item("AutoDiagonalNormal.loc", 100.0))
    

In [176]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

Parameter containing:
tensor([100.0000,  -2.4763,  -1.0711,  ...,  -2.4452,   4.6454,   1.5156],
       device='cuda:0', requires_grad=True)

In [177]:
original_param_store["AutoDiagonalNormal.loc"]

tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

## AFTER BITFLIP

In [178]:
changed = not torch.equal(pyro.get_param_store()["AutoDiagonalNormal.loc"], #AFTER 
                          original_param_store["AutoDiagonalNormal.loc"], #BEFORE
                          )
print("Weights changed:", changed)

Weights changed: True


In [ ]:
#changed = not torch.equal(model_cnn.conv1.weight.data, original_weights)
#print("Weights changed:", changed)

In [ ]:
# calculate roc auc score
from sklearn.metrics import roc_auc_score

def calculate_roc_auc_score(y_true, y_pred, num_classes):
    # Convert to one-hot encoding
    y_true_one_hot = np.eye(num_classes)[y_true]
    y_pred_one_hot = np.eye(num_classes)[y_pred]
    
    # Calculate ROC AUC for each class
    roc_auc_scores = []
    for i in range(num_classes):
        if np.sum(y_true_one_hot[:, i]) > 0:  # Check if the class is present
            roc_auc = roc_auc_score(y_true_one_hot[:, i], y_pred[:, i])
            roc_auc_scores.append(roc_auc)
        else:
            roc_auc_scores.append(np.nan)  # Class not present in the test set

    return np.array(roc_auc_scores)

In [ ]:
roc_auc_scores = calculate_roc_auc_score(all_labels, all_predictions, num_classes)
print(f"ROC AUC scores: {roc_auc_scores}")

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Example data
y_true = [0, 1, 2, 2, 1]  # true labels
y_score = [
    [0.8, 0.1, 0.1],
    [0.2, 0.6, 0.2],
    [0.1, 0.2, 0.7],
    [0.1, 0.3, 0.6],
    [0.2, 0.7, 0.1]
]  # predicted probabilities from model

y_true = np.array(all_labels)
y_score = np.array(all_probs)

# Binarize labels
n_classes = y_score.shape[1]
y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all 10 ROC curves
plt.figure(figsize=(8, 6))
colors = [
    'tab:blue',      # 1
    'tab:orange',    # 2
    'tab:green',     # 3
    'tab:red',       # 4
    'tab:purple',    # 5
    'tab:brown',     # 6
    'tab:pink',      # 7
    'tab:gray',      # 8
    'tab:olive',     # 9
    'tab:cyan'       # 10
]
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)],
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves for Each Class')
plt.legend(loc="lower right")
plt.grid()
plt.show()